<a href="https://colab.research.google.com/github/Jelli36/Fake-News-Prediction/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump, load
import PIL
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import linear_model, naive_bayes, ensemble, tree, svm, model_selection,\
                    preprocessing, feature_extraction, metrics, base, pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML/train (2).csv", delimiter= ';')
display(df)

,Unnamed: 0,title,text,label
0,0,Palestinians switch off Christmas lights in Be...,"RAMALLAH, West Bank (Reuters) - Palestinians s...",1
1,1,China says Trump call with Taiwan president wo...,BEIJING (Reuters) - U.S. President-elect Donal...,1
2,2,FAIL! The Trump Organization’s Credit Score W...,While the controversy over Trump s personal ta...,0
3,3,Zimbabwe military chief's China trip was norma...,BEIJING (Reuters) - A trip to Beijing last wee...,1
4,4,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...,There has never been a more UNCOURAGEOUS perso...,0
...,...,...,...,...
24348,24348,Mexico Senate committee OK's air transport dea...,MEXICO CITY (Reuters) - A key committee in Mex...,1
24349,24349,BREAKING: HILLARY CLINTON’S STATE DEPARTMENT G...,IF SHE S NOT TOAST NOW THEN WE RE IN BIGGER TR...,0
24350,24350,trump breaks from stump speech to admire beaut...,kremlin nato was created for agression \nruss...,0
24351,24351,NFL PLAYER Delivers Courageous Message: Stop B...,Dallas Cowboys star wide receiver Dez Bryant t...,0


In [4]:
df.isnull().sum() # Check for null values

,0
Unnamed: 0,0
title,0
text,0
label,0


In [5]:
# Check and drop duplicate values
print(f"Number of duplicates: {np.sum(df.duplicated())}")

df.drop_duplicates(inplace = True)

print(f"Number of duplicates after: {np.sum(df.duplicated())}")

Number of duplicates: 0
Number of duplicates after: 0


In [6]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True) # Drop index

In [7]:
df.isnull().sum() # Check for null values

,0
title,0
text,0
label,0


In [9]:
df.isna().sum() # Check for na values

,0
title,0
text,0
label,0


In [10]:
# Concatenate 'title' and 'text' into a single column called 'info'
df['info'] = df['title']+' '+df['text']
print(df['info'])

0        Palestinians switch off Christmas lights in Be...
1        China says Trump call with Taiwan president wo...
2         FAIL! The Trump Organization’s Credit Score W...
3        Zimbabwe military chief's China trip was norma...
4        THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...
                               ...                        
24348    Mexico Senate committee OK's air transport dea...
24349    BREAKING: HILLARY CLINTON’S STATE DEPARTMENT G...
24350    trump breaks from stump speech to admire beaut...
24351    NFL PLAYER Delivers Courageous Message: Stop B...
24352    NORDSTROM STOCK TAKES NOSEDIVE After Trump Twe...
Name: info, Length: 24353, dtype: object


In [11]:
df.drop(['title','text'], axis = 1, inplace = True) # Drop columns since all values are already in df['info']

In [12]:
df['info'] = df['info'].str.replace(r'[^a-zA-Z0-9\s]','', regex = True) # Remove special characters

In [13]:
df['info'].isnull().sum() # Check for null values

np.int64(0)

In [14]:
display(df)

,label,info
0,1,Palestinians switch off Christmas lights in Be...
1,1,China says Trump call with Taiwan president wo...
2,0,FAIL The Trump Organizations Credit Score Wil...
3,1,Zimbabwe military chiefs China trip was normal...
4,0,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...
...,...,...
24348,1,Mexico Senate committee OKs air transport deal...
24349,0,BREAKING HILLARY CLINTONS STATE DEPARTMENT GAV...
24350,0,trump breaks from stump speech to admire beaut...
24351,0,NFL PLAYER Delivers Courageous Message Stop Bl...


In [15]:
vectorizer = TfidfVectorizer(stop_words = 'english', max_features = 5000) #Remove stop words
X = vectorizer.fit_transform(df['info']) # Fit and transform the text data into a TF-IDF matrix
y = df['label']
print(X.shape, y.shape)

(24353, 5000) (24353,)


In [16]:
# Initialize models
lr = LogisticRegression(solver='liblinear')
kfold = KFold(n_splits=20, shuffle=True, random_state=42)

##Logistic Regression

In [17]:
# Initialize variables to keep track of the best score and corresponding model
best_score = 0
best_model = None

# Loop through each fold in K-Fold cross-validation
for train_index, test_index in kfold.split(X):

    # Split the data into training and testing sets based on the current fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Create a logistic regression model using the 'liblinear' solver
    model_lr = LogisticRegression(solver='liblinear')

    # Train the model on the training set
    model_lr.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model_lr.predict(X_test)

    # Calculate the accuracy of the model on the test set
    accuracy_lr = accuracy_score(y_test, y_pred)

    if accuracy_lr > best_score:
        best_score = accuracy_lr
        best_model = model_lr

print(f"Best Accuracy: {best_score:.3f}")

Best Accuracy: 0.983


In [18]:
def predict_user_input(user_input):
    # Remove punctuation from the user's input using regular expressions
    user_input = re.sub(r'[^\w\s]', '', user_input)

    # Transform the cleaned input into the same vectorized format used for training
    user_input_vectorized = vectorizer.transform([user_input])

    # Use the trained best model to make a prediction
    prediction = best_model.predict(user_input_vectorized)

    # Return the corresponding label based on the prediction result
    return "Real News" if prediction[0] == 0 else "Fake News"

In [19]:
# Prompt the user to enter a review (e.g., a news headline or article snippet)
user_input = input("Enter a review: ")

# Pass the user's input to the prediction function and print the result ("Real News" or "Fake News")
print(predict_user_input(user_input))

Enter a review: Milk came from cows
Real News


#KNN

In [20]:
knn = KNeighborsClassifier(n_neighbors=5)
kfold = KFold(n_splits=10, shuffle=True, random_state=40)
scores = cross_val_score(knn, X, y, cv=kfold)
print(f"Accuracy scores for each fold: {scores}")
print(f"Mean accuracy across all folds: {np.mean(scores)}")
print("Best accuracy (highest performing fold):", scores.max())

Accuracy scores for each fold: [0.636289   0.64860427 0.63341544 0.8476386  0.64435318 0.86078029
 0.64558522 0.86365503 0.6386037  0.64476386]
Mean accuracy across all folds: 0.7063688579588175
Best accuracy (highest performing fold): 0.8636550308008214


In [21]:
# Initialize variables to keep track of the best score and corresponding model
best_score = 0
best_model = None

# Loop through each fold in K-Fold cross-validation
for train_index, test_index in kfold.split(X):

    # Split the data into training and testing sets based on the current fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Create a knn classifier model with a n_neighbors of 5
    model_knn = KNeighborsClassifier(n_neighbors=5)

    # Train the model on the training set
    model_knn.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model_knn.predict(X_test)

    # Calculate the accuracy of the model on the test set
    accuracy_knn = accuracy_score(y_test, y_pred)

    if accuracy_knn > best_score:
        best_score = accuracy_knn
        best_model = model_knn

print(f"Best Accuracy: {best_score:.3f}")

Best Accuracy: 0.864


In [22]:
def predict_user_input(user_input):
    # Remove punctuation from the user's input using regular expressions
    user_input = re.sub(r'[^\w\s]', '', user_input)

    # Transform the cleaned input into the same vectorized format used for training
    user_input_vectorized = vectorizer.transform([user_input])

    # Use the trained best model to make a prediction
    prediction = best_model.predict(user_input_vectorized)

    # Return the corresponding label based on the prediction result
    return "Real News" if prediction[0] == 0 else "Fake News"

In [25]:
# Prompt the user to enter a review (e.g., a news headline or article snippet)
user_input = input("Enter a review: ")

# Pass the user's input to the prediction function and print the result ("Real News" or "Fake News")
print(predict_user_input(user_input))

Enter a review: The United Nations convened an emergency summit today in Geneva to address the rising number of people displaced due to extreme weather events.
Fake News
